In [5]:
import os
import tarfile
import xml.dom.minidom
import pandas as pd
from xml.etree import cElementTree as ET
import nltk
#nltk.download()
#import nltk.word_tokenize
#import metapy


In [6]:
df = pd.read_csv("timeSeriesPrices.csv")
#print(df)

In [7]:
docs = [] #(date, document_data)
for month in os.listdir("./documents"):
    for day in os.listdir("./documents/"+month):
        for doc in os.listdir("./documents/"+month+"/"+day):
            tree = ET.parse("./documents/"+month+"/"+day+"/"+doc)
            root = tree.getroot()
            
            for page in root.findall('body'):
                for page1 in page.findall('body.content'):
                    for page2 in page1.findall('block'):
                        for page3 in page2.findall('p'):
                            if page3.text != None:
                                docs.append((month+"/"+day+"/00", page3.text))                           

            

print(len(docs))
print(docs[1])

746234
('05/03/00', 'BLAUSTEIN-Phoebe Evseroff. May 2, 2000. Beloved wife of the late Irving H. Blaustein, M.D. Devoted mother of Janet (Michael) Kublin and dear grandmother of Heather Ilene Kublin. Graveside services 11 AM Thursday, Beth El Cemetery, Paramus, N.J.')


In [8]:
D = []
for doc in docs:
    gore = 0
    bush = 0
    if "Gore" in doc[1]: 
        gore += 1
        
    if "Bush" in doc[1]:
        bush += 1
        
    if gore > 0 or bush > 0:
        D.append(doc)
        
print(len(D))
print(D[0][1])

23931
In a speech pungent with tough talk about criminals, drug abusers and his Republican rival for president, Vice President Al Gore called today for a broad array of anticrime measures, among them mandatory drug testing of all prisoners and parolees.


In [9]:
import nltk; nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/peterwasala/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy



In [11]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [45]:
#i = 0
#for doc in D:
    #print(doc[1])
    #words = doc[1]
    #words = [re.sub('\S*@\S*\s?', '', sent) for sent in doc[1]]
    #words = [re.sub('\s+', '', sent) for sent in doc[1]]
    #words = [re.sub("\'", '', sent) for sent in doc[1]]
    #print(words)
    #i+=1
    #if(i == 2):
    #    break

In [12]:
def sent_to_words(sentences):
    #print(sentences)
    for sentence in sentences:
        #print(sentence[1])
        yield(gensim.utils.simple_preprocess(str(sentences[1]), deacc=True))  # deacc=True removes punctuations

In [13]:
# Step 8

data_words = list(sent_to_words(D))


In [18]:
# Step 9
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

/Users/peterwasala/opt/anaconda3/envs/py35/lib/python3.5/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['choosing', 'southern', 'state', 'as', 'the', 'backdrop', 'for', 'one', 'of', 'his', 'most', 'conservative', 'speeches', 'of', 'the', 'presidential', 'campaign', 'mr', 'gore', 'proposed', 'federal', 'spending', 'of', 'million', 'year', 'to', 'help', 'states', 'test', 'treat', 'and', 'counsel', 'prisoners', 'and', 'parolees', 'for', 'drug', 'use']


In [19]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
# Step 10
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [21]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
from spacy.cli import download
#print(download('en'))
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['choose', 'southern', 'state', 'backdrop', 'conservative', 'speech', 'presidential', 'campaign', 'mr', 'gore', 'propose', 'federal', 'spending', 'year', 'help', 'state', 'test', 'treat', 'counsel', 'prisoner', 'parole', 'drug']]


In [22]:
#Step 11
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
#print(corpus[:1])
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('backdrop', 1),
  ('campaign', 1),
  ('choose', 1),
  ('conservative', 1),
  ('counsel', 1),
  ('drug', 1),
  ('federal', 1),
  ('gore', 1),
  ('help', 1),
  ('mr', 1),
  ('parole', 1),
  ('presidential', 1),
  ('prisoner', 1),
  ('propose', 1),
  ('southern', 1),
  ('speech', 1),
  ('spending', 1),
  ('state', 2),
  ('test', 1),
  ('treat', 1),
  ('year', 1)]]

In [23]:
# Step 12
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [24]:
#Step 13
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.091*"state" + 0.045*"presidential" + 0.045*"parole" + 0.045*"treat" + '
  '0.045*"test" + 0.045*"spending" + 0.045*"speech" + 0.045*"southern" + '
  '0.045*"propose" + 0.045*"prisoner"'),
 (1,
  '0.091*"state" + 0.045*"presidential" + 0.045*"parole" + 0.045*"treat" + '
  '0.045*"test" + 0.045*"spending" + 0.045*"speech" + 0.045*"southern" + '
  '0.045*"propose" + 0.045*"prisoner"'),
 (2,
  '0.091*"state" + 0.045*"presidential" + 0.045*"parole" + 0.045*"treat" + '
  '0.045*"test" + 0.045*"spending" + 0.045*"speech" + 0.045*"southern" + '
  '0.045*"propose" + 0.045*"prisoner"'),
 (3,
  '0.091*"state" + 0.045*"presidential" + 0.045*"parole" + 0.045*"treat" + '
  '0.045*"test" + 0.045*"spending" + 0.045*"speech" + 0.045*"southern" + '
  '0.045*"propose" + 0.045*"prisoner"'),
 (4,
  '0.091*"state" + 0.045*"presidential" + 0.045*"parole" + 0.045*"treat" + '
  '0.045*"test" + 0.045*"spending" + 0.045*"speech" + 0.045*"southern" + '
  '0.045*"propose" + 0.045*"prisoner"'),
 (5,
  '0.

In [25]:
# Step 14
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -3.2752565433564347

Coherence Score:  0.9999999999999998


In [27]:
def generate_prior(significance):
    """ 
    Define a prior on the topic model parameters using significant terms and their impact values. 
    """
    

    #Step 4a
    pos_impact = [sig - .90 for sig in significance]
    neg_impact = [-sig - .90 for sig in significance]
    for i, sig in enumerate(pos_impact):
        if sig < 0:
            pos_impact[i] = 0
            
    for i, sig in enumerate(neg_impact):
        if sig < 0:
            neg_impact[i] = 0
        
    pos_impact = numpy.array(pos_impact)
    neg_impact = numpy.array(neg_impact)
    
    
    
    prior = []
    
    for i in range(len(pos_impact)):
        count_pos = np.count_nonzero(positive_sigs)
        count_neg = np.count_nonzero(negative_sigs)
        sum_impact = count_pos + count_neg
        if sum_impact != 0:
            prob = count_pos / sum_impact
            #step 4b
            if (prob > 0.9):
                prior.append(pos_impact / np.sum(pos_impact))
            elif (prob < 0.1):
                prior.append(neg_impact / np.sum(neg_impact))
            else:
                prior.append(pos_impact / np.sum(pos_impact))
                prior.append(neg_impact / np.sum(neg_impact))                
    
    if prior != None:
        return numpy.array(prior)

    return None

SyntaxError: invalid syntax (<ipython-input-27-2fae923079e4>, line 10)